In [1]:
import numpy as np
import lightgbm as lgb
from wideboost.wrappers import wlgb

import tensorflow as tf
import tensorflow_datasets as tfds

ds_train, ds_info = tfds.load(
    'titanic',
    split=['train'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

for i in ds_train[0].batch(60000):
    a = i

In [2]:
## Keep only numeric columns. Randomly split the dataset into training and test

X = []
for s in a[0].keys():
    if a[0][s].dtype != tf.string:
        X.append(a[0][s].numpy().reshape([-1,1]))

X = np.concatenate(X,axis=1)

idx = np.arange(X.shape[0])
np.random.seed(1)
train_idx = np.random.choice(idx, round(0.7*X.shape[0]), replace=False)
test_idx = np.setdiff1d(idx,train_idx)

xtrain = X[train_idx,:]
ytrain = a[1].numpy()[train_idx]

xtest = X[test_idx,:]
ytest = a[1].numpy()[test_idx]

train_data = lgb.Dataset(xtrain, label=ytrain)
test_data = lgb.Dataset(xtest, label=ytest)

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, space_eval

best_val = 1.0

def objective(param):
    global best_val
    #watchlist = [(dtrain,'train'),(dtest,'test')]
    ed1_results = dict()
    print(param)
    param['num_leaves'] = round(param['num_leaves']+1)
    param['min_data_in_leaf'] = round(param['min_data_in_leaf'])
    wbst = wlgb.train(param,
                      train_data,
                      num_boost_round=100,
                      valid_sets=test_data,
                      evals_result=ed1_results)
    output = min(ed1_results['valid_0']['binary_error'])
    
    if output < best_val:
        print("NEW BEST VALUE!")
        best_val = output
    
    return {'loss': output, 'status': STATUS_OK }

spc = {
    'btype': hp.choice('btype',['R','I','Rn','In']),
    'extra_dims': hp.choice('extra_dims',range(16)),
    'objective': hp.choice('objective',['binary']),
    'metric':hp.choice('metric',['binary_error']),
    'learning_rate': hp.loguniform('learning_rate', -7, 0),
    'num_leaves' : hp.qloguniform('num_leaves', 0, 7, 1),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
    'min_data_in_leaf': hp.qloguniform('min_data_in_leaf', 0, 6, 1),
    'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', -16, 5),
    'lambda_l1': hp.choice('lambda_l1', [0, hp.loguniform('lambda_l1_positive', -16, 2)]),
    'lambda_l2': hp.choice('lambda_l2', [0, hp.loguniform('lambda_l2_positive', -16, 2)])
}


best = fmin(objective,
    space=spc,
    algo=tpe.suggest,
    max_evals=100)

{'bagging_fraction': 0.9601405034485597, 'btype': 'In', 'extra_dims': 6, 'feature_fraction': 0.6167861530327529, 'lambda_l1': 3.4973799028357666, 'lambda_l2': 0.04429460908347433, 'learning_rate': 0.0014190821912160362, 'metric': 'binary_error', 'min_data_in_leaf': 44.0, 'min_sum_hessian_in_leaf': 3.144798121940943e-05, 'num_leaves': 10.0, 'objective': 'binary'}
Overwriting param `num_class`                          
Overwriting param `objective` while setting `fobj` in train.
Moving param `metric` to an feval.                     
[1]	valid_0's binary_error: 0.193384                   
[2]	valid_0's binary_error: 0.216285                   
[3]	valid_0's binary_error: 0.216285                   
[4]	valid_0's binary_error: 0.216285                   
[5]	valid_0's binary_error: 0.216285                   
[6]	valid_0's binary_error: 0.216285                   
[7]	valid_0's binary_error: 0.216285                   
[8]	valid_0's binary_error: 0.216285                   
[9]	valid_0's 

[18]	valid_0's binary_error: 0.208651                                             
[19]	valid_0's binary_error: 0.208651                                             
[20]	valid_0's binary_error: 0.208651                                             
[21]	valid_0's binary_error: 0.208651                                             
[22]	valid_0's binary_error: 0.208651                                             
[23]	valid_0's binary_error: 0.206107                                             
[24]	valid_0's binary_error: 0.208651                                             
[25]	valid_0's binary_error: 0.21374                                              
[26]	valid_0's binary_error: 0.208651                                             
[27]	valid_0's binary_error: 0.211196                                             
[28]	valid_0's binary_error: 0.211196                                             
[29]	valid_0's binary_error: 0.211196                                             
[30]

[8]	valid_0's binary_error: 0.16285                                               
[9]	valid_0's binary_error: 0.165394                                              
[10]	valid_0's binary_error: 0.155216                                             
[11]	valid_0's binary_error: 0.160305                                             
[12]	valid_0's binary_error: 0.160305                                             
[13]	valid_0's binary_error: 0.160305                                             
[14]	valid_0's binary_error: 0.157761                                             
[15]	valid_0's binary_error: 0.155216                                             
[16]	valid_0's binary_error: 0.155216                                             
[17]	valid_0's binary_error: 0.155216                                             
[18]	valid_0's binary_error: 0.155216                                             
[19]	valid_0's binary_error: 0.160305                                             
[20]

Overwriting param `num_class`                                                     
Overwriting param `objective` while setting `fobj` in train.                      
Moving param `metric` to an feval.                                                
[1]	valid_0's binary_error: 0.198473                                              
[2]	valid_0's binary_error: 0.195929                                              
[3]	valid_0's binary_error: 0.193384                                              
[4]	valid_0's binary_error: 0.19084                                               
[5]	valid_0's binary_error: 0.188295                                              
[6]	valid_0's binary_error: 0.188295                                              
[7]	valid_0's binary_error: 0.188295                                              
[8]	valid_0's binary_error: 0.188295                                              
[9]	valid_0's binary_error: 0.188295                                              
[10]

[96]	valid_0's binary_error: 0.185751                                             
[97]	valid_0's binary_error: 0.185751                                             
[98]	valid_0's binary_error: 0.185751                                             
[99]	valid_0's binary_error: 0.185751                                             
[100]	valid_0's binary_error: 0.185751                                            
{'bagging_fraction': 0.9433185610619872, 'btype': 'Rn', 'extra_dims': 1, 'feature_fraction': 0.5754585241267408, 'lambda_l1': 0.1370842624445247, 'lambda_l2': 0, 'learning_rate': 0.91010081741303, 'metric': 'binary_error', 'min_data_in_leaf': 94.0, 'min_sum_hessian_in_leaf': 0.01089040573518605, 'num_leaves': 12.0, 'objective': 'binary'}
Overwriting param `num_class`                                                     
Overwriting param `objective` while setting `fobj` in train.                      
Moving param `metric` to an feval.                                             

...

[75]	valid_0's binary_error: 0.160305                                              
[76]	valid_0's binary_error: 0.160305                                              
[77]	valid_0's binary_error: 0.160305                                              
[78]	valid_0's binary_error: 0.160305                                              
[79]	valid_0's binary_error: 0.160305                                              
[80]	valid_0's binary_error: 0.160305                                              
[81]	valid_0's binary_error: 0.160305                                              
[82]	valid_0's binary_error: 0.160305                                              
[83]	valid_0's binary_error: 0.157761                                              
[84]	valid_0's binary_error: 0.157761                                              
[85]	valid_0's binary_error: 0.157761                                              
[86]	valid_0's binary_error: 0.157761                                       

[65]	valid_0's binary_error: 0.175573                                              
[66]	valid_0's binary_error: 0.173028                                              
[67]	valid_0's binary_error: 0.173028                                              
[68]	valid_0's binary_error: 0.173028                                              
[69]	valid_0's binary_error: 0.175573                                              
[70]	valid_0's binary_error: 0.175573                                              
[71]	valid_0's binary_error: 0.175573                                              
[72]	valid_0's binary_error: 0.173028                                              
[73]	valid_0's binary_error: 0.173028                                              
[74]	valid_0's binary_error: 0.173028                                              
[75]	valid_0's binary_error: 0.173028                                              
[76]	valid_0's binary_error: 0.173028                                       

[55]	valid_0's binary_error: 0.21374                                               
[56]	valid_0's binary_error: 0.21374                                               
[57]	valid_0's binary_error: 0.21374                                               
[58]	valid_0's binary_error: 0.21374                                               
[59]	valid_0's binary_error: 0.21374                                               
[60]	valid_0's binary_error: 0.21374                                               
[61]	valid_0's binary_error: 0.21374                                               
[62]	valid_0's binary_error: 0.21374                                               
[63]	valid_0's binary_error: 0.21374                                               
[64]	valid_0's binary_error: 0.21374                                               
[65]	valid_0's binary_error: 0.21374                                               
[66]	valid_0's binary_error: 0.21374                                        

[45]	valid_0's binary_error: 0.19084                                               
[46]	valid_0's binary_error: 0.19084                                               
[47]	valid_0's binary_error: 0.19084                                               
[48]	valid_0's binary_error: 0.19084                                               
[49]	valid_0's binary_error: 0.195929                                              
[50]	valid_0's binary_error: 0.19084                                               
[51]	valid_0's binary_error: 0.19084                                               
[52]	valid_0's binary_error: 0.19084                                               
[53]	valid_0's binary_error: 0.188295                                              
[54]	valid_0's binary_error: 0.188295                                              
[55]	valid_0's binary_error: 0.185751                                              
[56]	valid_0's binary_error: 0.185751                                       

[35]	valid_0's binary_error: 0.201018                                              
[36]	valid_0's binary_error: 0.198473                                              
[37]	valid_0's binary_error: 0.198473                                              
[38]	valid_0's binary_error: 0.195929                                              
[39]	valid_0's binary_error: 0.198473                                              
[40]	valid_0's binary_error: 0.195929                                              
[41]	valid_0's binary_error: 0.195929                                              
[42]	valid_0's binary_error: 0.198473                                              
[43]	valid_0's binary_error: 0.203562                                              
[44]	valid_0's binary_error: 0.211196                                              
[45]	valid_0's binary_error: 0.21374                                               
[46]	valid_0's binary_error: 0.211196                                       

In [4]:
print(best_val)
print(space_eval(spc, best))

0.13486005089058525
{'bagging_fraction': 0.5131187471861871, 'btype': 'In', 'extra_dims': 13, 'feature_fraction': 0.6427181669100851, 'lambda_l1': 4.2453667594664e-07, 'lambda_l2': 0, 'learning_rate': 0.035666421079899376, 'metric': 'binary_error', 'min_data_in_leaf': 1.0, 'min_sum_hessian_in_leaf': 0.0025281313685325203, 'num_leaves': 15.0, 'objective': 'binary'}
